In [14]:
from tqdm import tqdm
import string

tu_ngoai_ngu = []

def contains_vietnamese(word):
    vietnamese_characters = set("àáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳỵỷỹý")
    non_vietnamese_characters = set("wzjWZJ")
    
    # Check if any Vietnamese or non-Vietnamese characters are present
    vietnamese_condition = any(char in vietnamese_characters for char in word.lower())
    non_vietnamese_condition = any(char in non_vietnamese_characters for char in word.lower())
    uppercase_condition = word.isupper()
    
    # Check if the consecutive characters "gia" are present at a fixed position
    gia_condition = "gia" in word.lower() and word.lower().find("gia") == 2  # Assuming the fixed position is index 2
    
    return vietnamese_condition or non_vietnamese_condition or uppercase_condition or gia_condition

def remove_punctuation(word):
    return ''.join(char for char in word if char not in string.punctuation)

def kiem_tra_so_am_tiet(tu):
    diphthongs = ["ai", "oi", "ay", "uy", "au","ao","oa", "gia", "eo", "ia", "ua", "ue", "iu", "io", "ua", "uo", "uô", "ươ", "ưa", "uơ", "ui","ia", "ie", "iu", "oi", "ôi", "ơi", "ua", "uô", "ươ", "uy", "uay", "uôy"]
    
    am_tiet = 0
    tu_lower = tu.lower()
    
    i = 0
    while i < len(tu_lower):
        if tu_lower[i:i+2] in diphthongs:
            am_tiet += 1
            i += 2
        elif tu_lower[i] in 'aeiouyáàạảãăắằặẳẵâấầậẩẫéèẹẻẽêếềệểễóòọỏõọôốồộổỗơớờợởỡíìịỉĩúùụủũưứừựửữýỳỵỷỹ':
            am_tiet += 1
            i += 1
        else:
            i += 1

    return am_tiet

def check_uppercase_count(word):
    uppercase_count = sum(1 for char in word if char.isupper())
    return uppercase_count

with open("test.txt", "r", encoding="utf-8") as file:
    data_pro = [line.replace('\n', '') for line in file]

# Build a list of foreign words
eng = []
for i in tqdm(range(len(data_pro))):
    # Handle hyphens by replacing with spaces
    cleaned_line = data_pro[i].replace('-', ' ')
    
    # Split the line into individual words
    word_list = cleaned_line.split(' ')
    
    for word in word_list:
        cleaned_word = remove_punctuation(word)

        if cleaned_word != '' and not contains_vietnamese(cleaned_word) and kiem_tra_so_am_tiet(cleaned_word) > 1 or any(char in 'wzjWZJ' for char in cleaned_word) or check_uppercase_count(cleaned_word) >= 2:
            tu_ngoai_ngu.append(cleaned_word)

# Save the list of filtered words to a separate file
with open("loc.txt", "w", encoding="utf-8") as filter:
    for word in tu_ngoai_ngu:
        filter.write(word + '\n')


100%|██████████| 28/28 [00:00<00:00, 1805.80it/s]


In [15]:
with open("tesst_e.txt", "w", encoding="utf-8") as data1:
    for i in tqdm(range(len(data_pro)), desc="Processing lines", unit="lines"):
        word_list = data_pro[i].split(' ')
        sen = ''
        for word in word_list:
            cleaned_word = remove_punctuation(word)
            if cleaned_word != '':
                original_word = word  # Lưu trữ từ gốc trước khi bị xử lý
                if cleaned_word in tu_ngoai_ngu:
                    # If the word is in tu_ngoai_ngu, replace it with "ENG_(index)"
                    index = tu_ngoai_ngu.index(cleaned_word)
                    sen = sen + "ENG_" + str(index) + ' '
                else:
                    sen = sen + original_word + ' '
        data1.write(sen + '\n')


Processing lines: 100%|██████████| 28/28 [00:00<00:00, 4692.37lines/s]


In [16]:
import re
import numpy as np
from unidecode import unidecode
typo = {"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}
vowel= {
    'a': ['à', 'á', 'ã', 'ạ'],
    'à': ['a', 'á', 'ã', 'ạ'],
    'á': ['à', 'a', 'ã', 'ạ'],
    'ã': ['à', 'á', 'a', 'ạ'],
    'ạ': ['à', 'á', 'ã', 'a'],
    'ă': ['ắ', 'ằ', 'ẵ', 'ặ'],
    'ắ': ['ă', 'ằ', 'ẵ', 'ặ'],
    'ằ': ['ă', 'ắ', 'ẵ', 'ặ'],
    'ẵ': ['ă', 'ắ', 'ằ', 'ặ'],
    'ặ': ['ă', 'ắ', 'ằ', 'ẵ'],
    'â': ['ấ', 'ầ', 'ẫ', 'ậ', 'ẩ'],
    'ấ': ['â', 'ầ', 'ẫ', 'ậ', 'ẩ'],
    'ầ': ['â', 'ấ', 'ẫ', 'ậ', 'ẩ'],
    'ẫ': ['â', 'ấ', 'ầ', 'ậ', 'ẩ'],
    'ậ': ['â', 'ấ', 'ầ', 'ẫ', 'ẩ'],
    'ẩ': ['â', 'ấ', 'ầ', 'ẫ', 'ậ'],
    'e': ['è', 'é', 'ẽ', 'ẹ'],
    'è': ['e', 'é', 'ẽ', 'ẹ'],
    'é': ['è', 'e', 'ẽ', 'ẹ'],
    'ẽ': ['è', 'e', 'é', 'ẹ'],
    'ẹ': ['è', 'e', 'é', 'ẽ'],
    'ê': ['ề', 'ế', 'ễ', 'ệ', 'ể'],
    'ề': ['ê', 'ế', 'ễ', 'ệ', 'ể'],
    'ế': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ễ': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ệ': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ể': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'i': ['ì', 'í', 'ĩ', 'ị'],
    'ì': ['i', 'í', 'ĩ', 'ị'],
    'í': ['ì', 'i', 'ĩ', 'ị'],
    'ĩ': ['ì', 'i', 'í', 'ị'],
    'ị': ['ì', 'i', 'í', 'ĩ'],
    'o': ['ò', 'ó', 'õ', 'ọ'],
    'ò': ['o', 'ó', 'õ', 'ọ'],
    'ó': ['o', 'ò', 'õ', 'ọ'],
    'õ': ['o', 'ò', 'ó', 'ọ'],
    'ọ': ['o', 'ò', 'ó', 'õ'],
    'ô': ['ồ', 'ố', 'ỗ', 'ộ', 'ổ'],
    'ồ': ['ô', 'ố', 'ỗ', 'ộ', 'ổ'],
    'ố': ['ô', 'ô', 'ỗ', 'ộ', 'ổ'],
    'ỗ': ['ô', 'ô', 'ồ', 'ộ', 'ổ'],
    'ộ': ['ô', 'ô', 'ồ', 'ố', 'ổ'],
    'ổ': ['ô', 'ô', 'ồ', 'ố', 'ỗ'],
    'ơ': ['ờ', 'ớ', 'ỡ', 'ợ', 'ở'],
    'ờ': ['ơ', 'ớ', 'ỡ', 'ợ', 'ở'],
    'ớ': ['ơ', 'ơ', 'ỡ', 'ợ', 'ở'],
    'ỡ': ['ơ', 'ơ', 'ờ', 'ợ', 'ở'],
    'ợ': ['ơ', 'ơ', 'ờ', 'ớ', 'ở'],
    'ở': ['ơ', 'ơ', 'ờ', 'ớ', 'ỡ'],
    'u': ['ù', 'ú', 'ũ', 'ụ'],
    'ù': ['u', 'ú', 'ũ', 'ụ'],
    'ú': ['u', 'ù', 'ũ', 'ụ'],
    'ũ': ['u', 'ù', 'ú', 'ụ'],
    'ụ': ['u', 'ù', 'ú', 'ũ'],
    'ư': ['ừ', 'ứ', 'ữ', 'ự', 'ử'],
    'ừ': ['ư', 'ứ', 'ữ', 'ự', 'ử'],
    'ứ': ['ư', 'ư', 'ữ', 'ự', 'ử'],
    'ữ': ['ư', 'ư', 'ừ', 'ự', 'ử'],
    'ự': ['ư', 'ư', 'ừ', 'ứ', 'ử'],
    'ử': ['ư', 'ư', 'ừ', 'ứ', 'ữ'],
    'y': ['ỳ', 'ý', 'ỹ', 'ỵ'],
    'ỳ': ['y', 'ý', 'ỹ', 'ỵ'],
    'ý': ['y', 'ỳ', 'ỹ', 'ỵ'],
    'ỹ': ['y', 'ỳ', 'ý', 'ỵ'],
    'ỵ': ['y', 'ỳ', 'ý', 'ỹ']
}

adjacent_keys = {
            'a': ['q', 'w', 's', 'z'],
            'b': ['v', 'g', 'h', 'n'],
            'c': ['x', 'd', 'f', 'v'],
            'd': ['s', 'e', 'r', 'f', 'c', 'x'],
            'e': ['w', 's', 'd', 'r'],
            'f': ['d', 'r', 't', 'g', 'v', 'c'],
            'g': ['f', 't', 'y', 'h', 'b', 'v'],
            'h': ['g', 'y', 'u', 'j', 'n', 'b'],
            'i': ['u', 'h', 'j', 'k', 'o'],
            'j': ['h', 'u', 'i', 'k', 'm', 'n'],
            'k': ['j', 'i', 'o', 'l', 'm'],
            'l': ['k', 'o', 'p'],
            'm': ['n', 'j', 'k'],
            'n': ['b', 'h', 'j', 'm'],
            'o': ['i', 'k', 'l', 'p'],
            'p': ['o', 'l'],
            'q': ['a', 'w', '1', '2'],
            'r': ['e', 'd', 'f', 't', '4','5'],
            's': ['a', 'w', 'e', 'd', 'f', 'x', 'z'],
            't': ['r', 'f', 'g', 'y'],
            'u': ['y', 'h', 'j', 'i'],
            'v': ['c', 'f', 'g', 'b'],
            'w': ['q', 'a', 's', 'e', 'd'],
            'x': ['z', 's', 'd', 'c'],
            'y': ['t', 'g', 'h', 'u'],
            'z': ['a', 's', 'x']
        }

acronym = {"không": ["ko", "k"], "anh": ["a"], "em": ["e"], "biết": ["bít", "bt"], "giờ": ["h"], "gì": ["j"], "muốn": ["mún"],
           "học": ["hc", "hok"], "làm": ["lm"], "yêu": ["iu"], "chồng": ["ck"], "vợ": ["vk"], "ông": ["ô"], "được": ["đc", "dc"],
           "tôi": ["t"], "bạn": ["b","bn", "pạn"], "nước": ["nc"], "không liên quan": ["klq"], "không hiểu sao": ["khs"], "chả hiểu sao": ["chs"],
           "rồi": ["òy", "oy", "r"], "vậy": ["v", "z"], "mới": ["ms"], "vẫn": ["vx"], "Không": ["Ko", "K"], "Anh": ["A"], "Em": ["E"],
           "Biết": ["Bít", "Bt"], "Giờ": ["H"], "Gì": ["J"], "Muốn": ["Mún"], "Học": ["Hc", "Hok"], "Làm": ["Lm"], "Yêu": ["Iu"], "Chồng": ["Ck"],
           "Vợ": ["Vk"], "Ông": ["Ô"], "Được": ["Đc", "Dc"], "Tôi": ["T"], "Bạn": ["B","Bn", "Pạn"], "Nước": ["Nc"], "Không Liên Quan": ["Klq"],
           "Không Hiểu Sao": ["Khs"], "Chả Hiểu Sao": ["Chs"], "Rồi": ["Òy", "Oy", "R"], "Vậy": ["V", "Z"], "Mới": ["Ms"], "Vẫn": ["Vx"], "Yếu": ["Íu"],
           "yếu": ["íu"], "Chiều": ["chìu"], "chiều": ["chìu"], "Tiêu": ["Tiu"], "tiêu": ["tiu"], "Tuổi": ["Tủi"], "tuổi": ["tủi"]}
teen = {"ch": "ck", "ph": "f", "th": "tk", "nh": "nk", "v": "d", "b": "p", "Ch": "Ck", "Ph": "F", "Th": "Tk", "Nh": "Nk", "V": "D", "B": "P","d": "z", "D":"Z"}
def teen_code(sentence, pivot):
    random = np.random.uniform(0,1,1)[0]
    new_sentence=str(sentence)
    if random>pivot:
        for word in acronym.keys():
            if re.search(word, new_sentence):
                random2 = np.random.uniform(0,1,1)[0]
                if random2 < 0.5:
                    # Chọn một phần tử ngẫu nhiên từ danh sách và thay thế
                    replacement = np.random.choice(acronym[word], 1)[0]
                    new_sentence = new_sentence.replace(word, replacement)
        for word in teen.keys(): 
            if re.search(word, new_sentence):
                random3 = np.random.uniform(0,1,1)[0]
                if random3 < 0.05:
                    new_sentence = new_sentence.replace(word, teen[word])
        return new_sentence
    else:
        return sentence

    

def add_noise(sentence, pivot1,pivot2):
    sentence=teen_code(sentence,0.5)
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        elif sentence[i] == 'E' and sentence[i+1] == 'N' and sentence[i+2] == 'G':
            noisy_sentence+=sentence[i]
            noisy_sentence+=sentence[i+1]
            noisy_sentence+=sentence[i+2]
            noisy_sentence+=sentence[i+3]
            i = i+4
            while (sentence[i].isnumeric()):
                noisy_sentence+=sentence[i]
                i += 1
            noisy_sentence+=sentence[i]
        else: 
            random = np.random.uniform(0,1,1)[0]   
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in typo.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.4:
                        noisy_sentence+=typo[sentence[i]]
                    elif random2<0.8:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in typo.keys():
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.6:
                        noisy_sentence+=typo[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys():
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=region[sentence[i]]
                    elif random4<0.85 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in adjacent_keys.keys():
                    random5=np.random.uniform(0,1,1)[0]
                    if random5<=0.6:
                        chosen_key = np.random.choice(adjacent_keys[sentence[i]], 1)[0]
                        noisy_sentence += chosen_key
                    elif random5<0.85:
                        noisy_sentence+=unidecode(sentence[i])
                    else: 
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random6=np.random.uniform(0,1,1)[0]
                        if random6<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.23:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.50:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass
      
        i += 1
    return noisy_sentence


In [17]:
with open('tesst_e.txt', 'r', encoding='utf-8') as f:
  data1 = f.readlines()
print('Number of sequences: ', len(data1))
print('First sequence: ', data1[10])

Number of sequences:  28
First sequence:  Từ Hán cổ và từ Hán Việt gọi chung là từ gốc Hán. 



In [18]:
from tqdm import tqdm
for i in tqdm(range(100)):
  print(add_noise(data1[i],0.93,0.96))

 28%|██▊       | 28/100 [00:00<00:00, 806.26it/s]

ENG_0 ENG_1 hay ENG_2 là một tổ chức quốc tế hoạt động phi lợi nhuật, pih chíhn ohủ và bao gồ các thành vien có trình đ chiyên gành. Tổ chc này hú trọng đến: tiêu chui, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành vjên và 65.000 thành viên cá nhân, ENG_2 bao gồm nững con ngưsi cụ thể trong cộng đồng ENG_0 Mọi chi tiết có thể tìm thấy taji ENG_5 cura ENG_2 

ENG_0 ENG_1 nằm ở gần thủ đ ENG_9 ENG_10 Hoa Kỳ và ENG_11 Thụy Sĩ. Số mội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nfánh của tổ chức tùy theo vij trí hoawjc sở thích. Hjện nay tổ chc có tới 90 chi nhánh trên toàn thế giới. 

Nhiệm ụ vaf mục đích hoạt động. 

Bảo đảm, cổ vũ cho sự phát trn, mở bộng và sử dụng ENG_0 được thuận lợi nhất cho mọi nguời trên toàn thế giới. 

Tiếng Việt, cũng yọi là tiếng Việt Nam hay Việt ngữ là ngôn ngữ của người iVệt và là ngôn ngữ chính tứhc tại Việt Nam. ây là tiếng m đẻ ủca khoảng 85% dân c Việt Nma cùng với hơn 4 t

IndexError: list index out of range